# Plongement lexical

L’expression renvoie à l’anglais *word embedding* qui décrit une méthode d’apprentissage à partir de la représentation en vecteurs de nombres réels des mots d’un lexique. Plus justement, on peut parler de vectorisation de mots.

L’idée est donc, à partir d’un texte, d’obtenir une liste de tokens qui sera représentée par des vecteurs de mêmes dimensions comme, par exemple :

$$
\vec{\text{autrement}} = \begin{pmatrix}
    0.8765   \\
    1.3567   \\
    \text{…} \\
    -0.6823
\end{pmatrix}
\hspace{2em}
\vec{\text{lune}} = \begin{pmatrix}
    0.2908   \\
    -1.4002  \\
    \text{…} \\
    0.1209
\end{pmatrix}
$$

Cette représentation n’aura de sens que dans l’espace vectoriel du texte. Ajoutons une phrase au texte, un commentaire au corpus de commentaires, et les vecteurs n’auront plus les mêmes valeurs.

On en revient toujours à l’importance de bien équilibrer un corpus.

## Word2Vec : la vectorisation de mots par Google

Word2Vec est en fait un ensemble de modèles développés par [Google](https://arxiv.org/abs/1301.3781) pour effectuer les tâches de vectorisation de mots. Ils prennent la forme de réseaux de neurones peu profonds, à deux couches, qui apprennent du contexte linguistique des mots afin d’en déduire une représentation vectorielle. L’intérêt de cette méthode est qu’elle suit l’hypothèse distributionnelle ([Harris 1954](https://doi.org/10.1080/00437956.1954.11659520)) résumée par John Rupert Firth en 1957 par la formule : « You shall know a word by the company it keeps. » En d’autres termes, des mots sémantiquement proches partageront un contexte similaire, ce qui, du point de vue mathématique, se traduit par des vecteurs voisins l’un de l’autre.

Deux architectures sont proposées :
- le modèle *skip-gram* ;
- le modèle sac de mot continu (*Continuous Bag-Of-Words*, ou CBOW).

### Le modèle *skip-gram*

Dans le modèle *skip-gram*, le réseau de neurones estime pour chaque mot du vocabulaire la probabilité qu’il soit dans le voisinage d’un mot particulier.

Prenons la phrase : *Le petit chat boit du lait.* Le modèle nous dira probablement que *petit* et *boit* ont de grandes chances d’accompagner le mot *chat*.

#### Architecture du modèle

L’architecture se présente sous la forme de deux couches :

![Schéma du modèle *skip-gram](./images/skip-gram-model.png)

Le vecteur d’entrée est soumis à la couche cachée, composée d’un certain nombre de neurones, puis envoyée à la couche de sortie, activée par une fonction *softmax* qui garantit d’abord pour chaque composante une valeur comprise entre 0 et 1 et ensuite pour l’ensemble une somme égale à 1.

La fonction *softmax* prend l’exponentielle de la composante d’un vecteur et la divise par la somme de toutes ses exponentielles.

#### Lors de la phase d’entraînement

Le modèle est paramétré pour une certaine fenêtre (i.e. : 2). Il parcourt le corpus d’entraînement et apprend depuis des paires de mots. Le vecteur d’entrée, décrit dans un certain nombre de dimensions (i.e. : 300), représente le mot en entrée, et le vecteur de sortie, décrit dans le même nombre de dimensions, le mot apparié.

Reprenons la phrase *Le petit chat boit du lait* avec une fenêtre d’apprentissage de deux. D’abord, le modèle s’intéresse au mot *Le* et à ses deux voisins du contexte droit, *petit* et *chat*. Il les apparie pour l’entraînement : (*Le*, *petit*) et (*Le*, *chat*). Il passe ensuite au mot suivant, *petit*, et cherche ses voisins dans la fenêtre spécifiée (*Le*, *chat*, *boit*) puis les apparie de nouveau : (*petit*, *Le*), (*petit*, *chat*) et (*petit*, *boit*).

#### Lors de la phase d’évaluation

De la taille du vocabulaire, le vecteur d’entrée, au format *one-hot*, est alors composé d’une unique composante à 1 pour le mot interrogé (par exemple le mot *chat*), les autres restant fixées à 0. La couche de sortie présente quant à elle un vecteur de probabilités de la taille du vocabulaire.

### Le modèle CBOW

Le modèle CBOW (*Continuous Bag of words*) est l’exacte symmétrie du modèle *skip-gram* : pour un contexte donné, il va estimer quel est le mot du vocabulaire qui a le plus de chance d’être au centre.

Pour une fenêtre de 2, considérant le contexte *Le*, *chat* et *boit*, on s’attend à ce que le modèle nous propose en sortie le mot *petit*.

#### Architecture du modèle

![Le modèle CBOW](./images/cbow-model.png)

#### Lors de la phase d’entraînement

L’appariement est différent du modèle *skip-gram*. Ici, avec le modèle CBOW, il prend en entrée les vecteurs qui décrivent le contexte, un vecteur par mot décrit dans un certain nombre de dimensions (i.e. : 300) et, en sortie, le vecteur du mot au centre.

Toujours pour la phrase *Le petit chat boit du lait* avec une fenêtre de 2, le modèle va d’abord apprendre de la paire ((*petit*, *chat*), *Le*), puis ((*Le*, *chat*, *boit*), *petit*), ensuite ((*Le*, *petit*, *boit*, *du*), *chat*) et ainsi de suite.

#### Lors de la phase d’évaluation

Le vecteur d’entrée est un *one-hot* avec des 1 pour les mots faisant partie du contexte et le vecteur de sortie sera une liste de probabilités, de la taille du vocabulaire, pour que le mot soit celui au centre.

## Entraîner un modèle Word2Vec

Sans entrer dans les détails de l’apprentissage automatique, Word2Vec aura besoin en entrée d’un corpus nettoyé, constitué d’une liste de phrases découpées en mots. Le fichier *bow-salammbo.pickle* contient un sac de mots du roman *Salammbô* de Flaubert ayant subi quelques traitements (bas de casse, mots vides retirés) :

In [ ]:
import pickle

with open('./data/bow-salammbo.pickle', 'rb') as f:
    bow = pickle.load(f)

En python, la librairie *Gensim* se charge d’implémenter les algorithmes Word2Vec :

In [ ]:
from gensim.models import word2vec

Nous aurons besoin de déterminer quelques paramètres :

- `vector_size` pour la taille du vecteur de mot. En théorie, plus un mot est décrit par un grand nombre de composantes, meilleur il sera défini. En pratique, on choisira entre 100 et 1000 composantes, le gain étant infime au-delà. À titre d’exemple, les équipes de Google à l’origine de Word2Vec ont établi des vecteurs à 300 composantes.
- `window` pour la fenêtre des contextes gauche et droit.
- `min_count` pour définir une fréquence d’occurrences minimale.
- `workers` pour déterminer le nombre de fils d’exécution à utiliser au moment de l’apprentissage.
- `sg` pour le modèle d’apprentissage (`1` pour *skip-gram*; CBOW autrement).

In [ ]:
# a CBOW model
model = word2vec.Word2Vec(
    sentences=bow,
    vector_size=100,
    window=5,
    min_count=3,
    workers=4,
    sg=0
)

### Estimations

La classe `wv` permet d’obtenir ensuite la description d’un vecteur :

In [ ]:
model.wv['cité']

Une méthode de la classe, `.most_similar()`, permet quant à elle d’obtenir les mots similaires.

In [ ]:
model.wv.most_similar('cité')